This file builds up several traditional language models on corpus of Wikipedia articles.

WikiText-2, a corpus of high-quality Wikipedia articles. The dataset was originally introduced in the following paper: https://arxiv.org/pdf/1609.07843v1.pdf. A raw version of the data can easily be viewed here: https://github.com/pytorch/examples/tree/master/word_language_model/data/wikitext-2.

The code was primarily designed to run on Google Colab.

In [2]:
!pip install torchdata==0.5.1
!pip install torchtext==0.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 32.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
  Attempting uninstall: torchdata
    Found existing installation: torchdata 0.7.0
    Uninstalling torchdata-0.7.0:
      Successfully uninstalled torchdata-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torcha

## Preprocessing Data

* <b>Sentence splitting:</b>&nbsp;&nbsp;&nbsp;&nbsp; We identify individual sentences by splitting paragraphs at punctuation tokens (".",  "!",  "?").

* <b>Sentence markers:</b>&nbsp;&nbsp;&nbsp;&nbsp;For both training and testing corpora, each sentence must be surrounded by a start-of-sentence (`<s>`) and end-of-sentence marker (`/s`).

* <b>Unknown words:</b>&nbsp;&nbsp;&nbsp;&nbsp;The WikiText dataset has replaced all works that are not in the corpora as (`<UNK>`)

In [3]:
START = "<s>"   # Start-of-sentence token
END = "</s>"    # End-of-sentence-token
UNK = "<UNK>"   # Unknown word token

In [4]:
import torchtext
import random
import sys

def preprocess(data, vocab=None):
    final_data = []
    lowercase = "abcdefghijklmnopqrstuvwxyz"
    for paragraph in data:
        paragraph = [x if x != '<unk>' else UNK for x in paragraph.split()]
        if vocab is not None:
            paragraph = [x if x in vocab else UNK for x in paragraph]
        if paragraph == [] or paragraph.count('=') >= 2: continue
        sen = []
        prev_punct, prev_quot = False, False
        for word in paragraph:
            if prev_quot:
                if word[0] not in lowercase:
                    final_data.append(sen)
                    sen = []
                    prev_punct, prev_quot = False, False
            if prev_punct:
                if word == '"':
                    prev_punct, prev_quot = False, True
                else:
                    if word[0] not in lowercase:
                        final_data.append(sen)
                        sen = []
                        prev_punct, prev_quot = False, False
            if word in {'.', '?', '!'}: prev_punct = True
            sen += [word]
        if sen[-1] not in {'.', '?', '!', '"'}: continue # Prevent a lot of short sentences
        final_data.append(sen)
    vocab_was_none = vocab is None
    if vocab is None:
        vocab = set()
    for i in range(len(final_data)):
        final_data[i] = [START] + final_data[i] + [END]
        if vocab_was_none:
            for word in final_data[i]:
                vocab.add(word)
    return final_data, vocab

def getDataset():
    dataset = torchtext.datasets.WikiText2(root='.data', split=('train', 'valid'))
    train_dataset, vocab = preprocess(dataset[0])
    test_dataset, _ = preprocess(dataset[1], vocab)

    return train_dataset, test_dataset

if __name__ == '__main__':
    train_dataset, test_dataset = getDataset()

In [5]:
if __name__ == '__main__':
    for x in random.sample(train_dataset, 10):
        print (x)

['<s>', 'His', 'wife', 'Connie', ',', 'whom', 'he', 'had', 'known', 'since', 'childhood', ',', 'ran', 'the', 'business', 'in', 'his', 'absence', '.', '</s>']
['<s>', '<UNK>', '<UNK>', ',', 'a', 'member', 'of', 'Anonymous', ',', 'commented', 'to', '<UNK>', 'about', 'the', 'Church', 'of', 'Scientology', "'s", 'denial', 'of', 'its', '"', 'Fair', 'Game', '"', 'policy', ':', '"', 'Even', 'if', 'the', 'name', "'", 'fair', 'game', "'", 'is', 'not', 'in', 'use', ',', 'the', 'Church', 'of', 'Scientology', 'is', 'an', 'organisation', 'that', 'continues', 'to', 'practice', 'a', 'vicious', 'policy', 'of', '<UNK>', 'against', 'perceived', 'enemies', ',', 'and', 'it', 'teaches', 'its', 'members', 'that', 'extreme', 'measures', 'are', 'morally', 'justified', 'if', 'they', 'aid', 'the', 'Church', '.', '"', '</s>']
['<s>', 'In', 'general', ',', 'this', 'ordering', 'is', 'not', 'unique', ';', 'a', 'DAG', 'has', 'a', 'unique', 'topological', 'ordering', 'if', 'and', 'only', 'if', 'it', 'has', 'a', 'direc

## The language moedel class

4 tpyes of language models are implemented: a unigram model, a smoothed uigram model, a bigram model, a smoothed bigram model.

In [6]:
import math
import random
from collections import defaultdict

class LanguageModel(object):
    def __init__(self, trainCorpus):
        '''
        Initialize and train the model (i.e. estimate the model's underlying probability
        distribution from the training corpus.)
        '''

        return

    def generateSentence(self):
        '''
        Generate a sentence by drawing words according to the model's probability distribution.
        Note: Think about how to set the length of the sentence in a principled way.
        '''

        raise NotImplementedError("Implement generateSentence in each subclass.")

    def getSentenceLogProbability(self, sentence):
        '''
        Calculate the log probability of the sentence provided.
        '''

        raise NotImplementedError("Implement getSentenceProbability in each subclass.")

    def getCorpusPerplexity(self, testCorpus):
        '''
        Calculate the perplexity of the corpus provided.
        '''

        raise NotImplementedError("Implement getCorpusPerplexity in each subclass.")

    def printSentences(self, n):
        '''
        Prints n sentences generated by your model.
        '''

        for i in range(n):
            sent = self.generateSentence()
            prob = self.getSentenceLogProbability(sent)
            print('Log Probability:', prob , '\tSentence:',sent)

### Unigram Model

In [7]:
from binascii import Error
class UnigramModel(LanguageModel):
    def __init__(self, trainCorpus):
        '''
        Initialize and train the model (i.e. estimate the model's underlying probability
        distribution from the training corpus.)
        'trainCorpus' is a list of sentence where each sentence is a list of words.
        '''
        self.corpus = trainCorpus
        self.model = self.train()


    def train(self):
        '''
        Train the model by calculating the probability of each word in then corpus.
        Return a dictionary which map tokens to their unigram frequency and the total counts of all tokens.
        '''
        model = {}
        total_counts = 0
        for sentence in self.corpus:
            for word in sentence:
                if word != '<s>':
                    total_counts += 1
                    if word in model:
                        model[word] += 1
                    else:
                        model[word] = 1

        for word in model:
            model[word] /= total_counts

        return model


    def generateSentence(self):
        '''
        Generate a sentence using the unigram model, the sentence should be starting with the '<s>' and end with the token '</s>'.
        Return a list of tokens representing the sentence.
        '''
        import random
        # Initialize the sentence
        sentence = []
        sentence.append('<s>')
        # Random select tokens from the dictionary based on the count (which is propotional to the frequency)
        while True:
            word = random.choices(list(self.model.keys()), weights = self.model.values())[0]
            sentence.append(word)
            if word == '</s>':
                break

        return sentence


    def getSentenceLogProbability(self, sentence):
        '''
        Calculate the log probability of the given sentence.
        Input is a list of tokens representing a sentence beginning with '<s>' and ends with '</s>'.
        Output is a float number representing the log probabity of the sentence from the unigram model.
        '''
        import math
        sum_logprob = 0
        for word in sentence[1:]:
            if word not in self.model:
                raise ValueError("Sentence contains tokens that doesn't belong to the corpus")
            sum_logprob += math.log(self.model[word])

        return sum_logprob

    def getCorpusPerplexity(self, testCorpus):
        '''
        Calculate the perplexity of the corpus provided. The begnning token '<s>' is not counted in this case
        The input is the corpus, which is list of sentences
        The output is the perplexity of the corpus
        '''
        import math
        total_log_prob = 0
        total_word_count = 0
        for sentence in testCorpus:
            for word in sentence[1:]:
                total_log_prob += math.log(self.model[word])
                total_word_count += 1
        return math.exp(-total_log_prob / total_word_count)


Here's sanity check on the function

In [8]:
def sanityCheck(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}

    #	Read in the test corpus
    train_corpus = ["By the Late Classic , a network of few <unk> ( few <unk> ) linked various parts of the city , running for several kilometres through its urban core .",
    "Few people realize how difficult it was to create Sonic 's graphics engine , which allowed for the incredible rate of speed the game 's known for ."]
    test_corpus = ["Classic few parts of the game allowed for few <unk> <unk> incredible city .",
                   "Few <unk> realize the difficult network , which linked the game to Sonic ."]
    train_corpus, _ = preprocess(train_corpus)
    test_corpus, _ = preprocess(test_corpus)
    sentence = preprocess(["Sonic was difficult ."])[0][0]

    # Correct answers
    if model_type == "unigram":
       senprobs = [-19.08542845, -114.5001481799, -108.7963657053, -53.6727664115, -55.4645258807]
       trainPerp, testPerp = 41.3308239726, 38.0122981569
       model = UnigramModel(train_corpus)
    elif model_type == "smoothed-unigram":
       senprobs = [-19.0405293515, -115.3479413049, -108.9114348746, -54.8190029616, -55.8122547346]
       trainPerp, testPerp = 41.9994393615, 39.9531928383
       model = SmoothedUnigramModel(train_corpus)
    elif model_type == "bigram":
       senprobs = [-float('inf'), -10.3450917073, -9.2464794186, -float('inf'), -float('inf')]
       trainPerp, testPerp = 1.3861445461, float('inf')
       model = BigramModel(train_corpus)
    elif model_type == "smoothed-bigram":
       senprobs = [-16.355820202, -76.0026113319, -74.2346475108, -47.2885760372, -51.2730261907]
       trainPerp, testPerp = 12.2307627397, 26.7193157699
       model = SmoothedBigramModelAD(train_corpus)
    else: assert False, 'Invalid model_type'

    print("--- TEST: generateSentence() ---")
    modelSen = model.generateSentence()
    senTestPassed = isinstance(modelSen, list) and len(modelSen) > 1 and isinstance(modelSen[0], str)
    if senTestPassed:
        print ("Test generateSentence() passed!")
    else:
        print ("Test generateSentence() failed; did not return a list of strings...")

    print("\n--- TEST: getSentenceLogProbability(...) ---")
    sentences = [sentence, *train_corpus, *test_corpus]
    failed = 0
    for i in range(len(sentences)):
        sen, correct_prob = sentences[i], senprobs[i]
        prob = round(model.getSentenceLogProbability(sen), 10)
        print("Correct log prob.:", correct_prob, '\tYour log prob.:', prob, '\t', 'PASSED' if prob == correct_prob else 'FAILED', '\t', sen)
        if prob != correct_prob: failed+=1

    if not failed:
        print ("Test getSentenceProbability(...) passed!")
    else:
        print("Test getSentenceProbability(...) failed on", failed, "sentence" if failed == 1 else 'sentences...')

    print("\n--- TEST: getCorpusPerplexity(...) ---")
    train_perp = round(model.getCorpusPerplexity(train_corpus), 10)
    test_perp = round(model.getCorpusPerplexity(test_corpus), 10)

    print("Correct train perp.:", trainPerp, '\tYour train perp.:', train_perp, '\t', 'PASSED' if trainPerp == train_perp else 'FAILED')
    print("Correct test perp.:", testPerp, '\tYour test perp.:', test_perp, '\t', 'PASSED' if testPerp == test_perp else 'FAILED')
    train_passed, test_passed = train_perp == trainPerp, test_perp == testPerp
    if train_passed and test_passed:
        print("Test getCorpusPerplexity(...) passed!")
    else:
        print("Test getCorpusPerplexity(...) failed on", "the training corpus and the testing corpus..." if not train_passed and not test_passed else "the testing corpus..." if not test_passed else "the training corpus...")

if __name__=='__main__':
    sanityCheck('unigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -19.08542845 	Your log prob.: -19.08542845 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -114.5001481799 	Your log prob.: -114.5001481799 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -108.7963657053 	Your log prob.: -108.7963657053 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -53.6727664115 	Your log prob.: -53.6727664115 	 PASSED 	 ['<s>', 'Classic', 'few', '

Sanity check with excuting time

In [10]:
import time

def sanityCheckFullDataset(model_type):
    model = UnigramModel(train_dataset)
    idxes = list(range(75,7500, 800))
    small_test_corpus = [test_dataset[idx] for idx in idxes]
    if model_type == 'unigram':
        senprobs = [-80.7782190984, -174.4769654449, -136.455148267, -225.5890741503, -719.0142129846, -236.350443633, -126.0056604204, -47.3424655612, -47.7775372096, -138.8159941929]
        testPerp = 881.0132848704
        model = UnigramModel(train_dataset)
    elif model_type == 'smoothed-unigram':
        senprobs = [-80.8423009715, -174.5131424172, -136.3181234818, -225.357454098, -719.1543898871, -236.6682968913, -126.1965419509, -47.4369338195, -47.7692144935, -138.542462715]
        testPerp = 881.6105352831
        model = SmoothedUnigramModel(train_dataset)
    elif model_type == 'bigram':
        senprobs = [-float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -32.1502020637, -float('inf'), -float('inf')]
        testPerp = float ('inf')
        model = BigramModel(train_dataset)
    elif model_type == 'smoothed-bigram':
        senprobs = [-61.3754065648, -141.9754903887, -107.0849366076, -168.4944718788, -619.9409055374, -195.8159911677, -86.3762008156, -32.4764801981, -48.124714509, -124.687107856]
        testPerp = 261.4247123506
        model = SmoothedBigramModelAD(train_dataset)
    else: assert False, 'Invalid model_type'
    print("\n--- TEST: getSentenceLogProbability(...) ---")
    failed = 0
    for i in range(len(small_test_corpus)):
        sen, correct_prob = small_test_corpus[i], senprobs[i]
        prob = round(model.getSentenceLogProbability(sen), 10)
        print("Correct log prob.:", correct_prob, '\tYour log prob.:', prob, '\t', 'PASSED' if prob == correct_prob else 'FAILED', '\t', sen)
        if prob != correct_prob: failed+=1

    if not failed:
        print ("Test getSentenceProbability(...) passed!")
    else:
        print("Test getSentenceProbability(...) failed on", failed, "sentence" if failed == 1 else 'sentences...')

    print("\n--- TEST: getCorpusPerplexity(...) ---")
    test_perp = round(model.getCorpusPerplexity(small_test_corpus), 10)

    print("Correct test perp.:", testPerp, '\tYour test perp.:', test_perp, '\t', 'PASSED' if testPerp == test_perp else 'FAILED')
    test_passed = test_perp == testPerp
    if test_passed:
        print("Test getCorpusPerplexity(...) passed!")
    else:
        print("Test getCorpusPerplexity(...) failed on the testing corpus...")

if __name__=='__main__':
    start_time = time.time()
    sanityCheckFullDataset('unigram')
    end_time = time.time()
    print(f'Execution time: {end_time - start_time} seconds')


--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -80.7782190984 	Your log prob.: -80.7782190984 	 PASSED 	 ['<s>', 'He', 'was', '<UNK>', 'at', '<UNK>', 'College', ',', 'Hobart', ',', 'and', '<UNK>', 'in', '1932', '.', '</s>']
Correct log prob.: -174.4769654449 	Your log prob.: -174.4769654449 	 PASSED 	 ['<s>', 'Despite', 'being', 'a', 'rare', 'Grade', '9', 'player', 'on', 'the', 'senior', 'team', ',', 'he', 'was', 'one', 'of', 'the', 'Knights', "'", 'two', 'leading', 'rushers', 'that', 'year', '.', '</s>']
Correct log prob.: -136.455148267 	Your log prob.: -136.455148267 	 PASSED 	 ['<s>', 'Burke', "'s", 'total', 'was', 'a', 'school', 'record', 'for', 'the', 'Big', 'Ten', 'Conference', 'Men', "'s", 'Basketball', 'Tournament', '.', '</s>']
Correct log prob.: -225.5890741503 	Your log prob.: -225.5890741503 	 PASSED 	 ['<s>', 'The', 'route', 'turns', 'to', 'the', 'northeast', ',', 'passing', 'near', 'the', '<UNK>', 'Leaf', 'Lakes', 'residential', 'development', ',', 'bef

Train model on the wikitext corpus

In [11]:
def runModel(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}
    # Read the corpora
    if model_type == 'unigram':
        model = UnigramModel(train_dataset)
    elif model_type == 'bigram':
        model = BigramModel(train_dataset)
    elif model_type == 'smoothed-unigram':
        model = SmoothedUnigramModel(train_dataset)
    else:
        model = SmoothedBigramModelAD(train_dataset)

    print("--------- 5 sentences from your model ---------")
    model.printSentences(5)

    print ("\n--------- Corpus Perplexities ---------")
    print ("Training Set:", model.getCorpusPerplexity(train_dataset))
    print ("Testing Set:", model.getCorpusPerplexity(test_dataset))

if __name__=='__main__':
    runModel('unigram')

--------- 5 sentences from your model ---------
Log Probability: -181.94414694213157 	Sentence: ['<s>', 'whose', 'narrates', 'of', 'and', 'explore', 'Gray', ',', 'of', 'was', 'treat', ',', 'Madrid', 'given', 'NME', ')', 'operated', 'the', ',', 'The', 'in', 'westward', '41', 'to', 'time', 'within', 'the', '</s>']
Log Probability: -518.4968791802588 	Sentence: ['<s>', 'xenon', 'halfway', '"', 'than', 'cede', '.', 'about', 'while', '1', '@,@', 'upon', 'and', 'the', 'with', 'majority', 'eleven', ',', 'in', 'parents', 'finally', 'order', 'Carolina', 'Moravec', 'January', 'with', '"', 'the', '@-@', 'Nathan', 'that', 'a', 'bit', 'him', 'the', '@-@', 'open', 'his', 'moved', 'the', 'placed', 'the', 'Army', 'capable', 'battalions', 'and', '<UNK>', 'from', '1975', 'early', 'acceptance', 'train', ',', 'reach', 'the', 'didn', 'play', 'km', 'feather', 'in', 'of', 'a', 'Second', ',', 'most', 'experience', 'would', 'to', 'York', 'tying', 'includes', 'legal', '@-@', '.', '.', '</s>']
Log Probability: -

### Smoothed Unigram Model

In [12]:
class SmoothedUnigramModel(UnigramModel):
    def train(self):
        '''
        Train the model by calculating the probability of each word in then corpus, with the smoothed method
        Return a dictionary which map tokens to their unigram frequency and the total counts of all tokens.
        '''
        model = {}
        total_counts = 0
        for sentence in self.corpus:
            for word in sentence:
                if word != '<s>':
                    total_counts += 1
                    if word in model:
                        model[word] += 1
                    else:
                        model[word] = 1

        number_tokens = len(model.keys())

        for word in model:
            model[word] = (model[word] + 1) / (total_counts + number_tokens)

        return model

In [13]:
if __name__=='__main__':
    sanityCheck('smoothed-unigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -19.0405293515 	Your log prob.: -19.0405293515 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -115.3479413049 	Your log prob.: -115.3479413049 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -108.9114348746 	Your log prob.: -108.9114348746 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -54.8190029616 	Your log prob.: -54.8190029616 	 PASSED 	 ['<s>', 'Classic', 'few

In [14]:
if __name__=='__main__':
    sanityCheckFullDataset('smoothed-unigram')


--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -80.8423009715 	Your log prob.: -80.8423009715 	 PASSED 	 ['<s>', 'He', 'was', '<UNK>', 'at', '<UNK>', 'College', ',', 'Hobart', ',', 'and', '<UNK>', 'in', '1932', '.', '</s>']
Correct log prob.: -174.5131424172 	Your log prob.: -174.5131424172 	 PASSED 	 ['<s>', 'Despite', 'being', 'a', 'rare', 'Grade', '9', 'player', 'on', 'the', 'senior', 'team', ',', 'he', 'was', 'one', 'of', 'the', 'Knights', "'", 'two', 'leading', 'rushers', 'that', 'year', '.', '</s>']
Correct log prob.: -136.3181234818 	Your log prob.: -136.3181234818 	 PASSED 	 ['<s>', 'Burke', "'s", 'total', 'was', 'a', 'school', 'record', 'for', 'the', 'Big', 'Ten', 'Conference', 'Men', "'s", 'Basketball', 'Tournament', '.', '</s>']
Correct log prob.: -225.357454098 	Your log prob.: -225.357454098 	 PASSED 	 ['<s>', 'The', 'route', 'turns', 'to', 'the', 'northeast', ',', 'passing', 'near', 'the', '<UNK>', 'Leaf', 'Lakes', 'residential', 'development', ',', 'bef

In [15]:
if __name__=='__main__':
    runModel('smoothed-unigram')

--------- 5 sentences from your model ---------
Log Probability: -142.36721236811863 	Sentence: ['<s>', '"', 'died', 'Dunster', '.', 'sniping', 'are', 'and', 'New', 'stayed', 'March', 'flotilla', 'corn', 'the', 'two', 'is', 'of', 'was', 'entire', 'the', '<UNK>', '</s>']
Log Probability: -281.15924546428084 	Sentence: ['<s>', 'command', '(', ',', 'season', 'the', ',', 'two', 'errors', 'well', 'brought', '5', 'Cruiser', 'Crush', 'cities', 'notes', 'presenter', 'economics', '"', '@-@', 'rest', 'number', 'Britons', 'started', 'sub', 'a', 'time', 'free', 'an', 'mass', 'Banai', 'series', 'a', 'the', 'instead', 'what', 'from', '</s>']
Log Probability: -431.3093453247379 	Sentence: ['<s>', 'for', '.', 'region', 'the', '<UNK>', 'just', 'as', 'Rapids', 'of', '<UNK>', 'before', '00', 'frequently', 'one', '<UNK>', 'particularly', '467', 'and', 'sic', '<UNK>', 'grandfather', 'the', 'period', 'rest', ',', 'Loch', 'research', '.', 'with', '<UNK>', 'then', 'a', 'the', 'of', 'City', 'resignation', ',',

### Bigram Model

In [16]:
class BigramModel(LanguageModel):
    def __init__(self, trainCorpus):
        '''
        Initialize and train the model
        Input 'trainCorpus' is a list of sentence where each sentence is a list of words.
        '''
        self.corpus = trainCorpus
        self.model = self.train()

    def train(self):
        '''
        Train the model by calculating the conditional probability of each word in then corpus.
        Return a dictionary of dictionaries stroing the conditional frequency of each word on the previous word
        '''
        model = {}
        for sentence in self.corpus:
            for i in range(len(sentence) - 1):
                word = sentence[i]
                word_next = sentence[i + 1]
                if word in model:
                    if word_next in model[word]:
                        model[word][word_next] += 1
                    else:
                        model[word][word_next] = 1
                else:
                    model[word] = {word_next : 1}

        # Calculate the conditional porbabilities
        for word in model:
            total_next_words = sum(model[word].values())
            for word_next in model[word]:
                model[word][word_next] /= total_next_words

        return model

    def generateSentence(self):
        '''
        Generate sentences based on the bigram model, the sentence starts with the token '<s>' and ends with the token '</s>'
        The output sentence is a list of tokens from the model beginning with '<s>' and ends with '</s>'
        '''
        import random

        # Initialize the sentence
        sentence = []
        sentence.append('<s>')

        curr_word = sentence[0] # Set the current word to be the '<s>'
        while True:
            next_word = random.choices(list(self.model[curr_word].keys()), weights=self.model[curr_word].values())[0]
            sentence.append(next_word)
            curr_word = next_word
            if curr_word == '</s>':
                break # Break if the token generated is '</s>'

        return sentence

    def getSentenceLogProbability(self, sentence):
        '''
        Calculate the conditonal log probability of the sentence
        Input is the list of tokens
        Output is the float number that is the log probability of the sentence
        '''
        import math
        log_prob_sum = 0
        for i in range(len(sentence) - 1):
            curr_word = sentence[i]
            next_word = sentence[i+1]
            if curr_word in self.model and next_word in self.model[curr_word]:
                log_prob_sum += math.log(self.model[curr_word][next_word])
            else:
                log_prob_sum = -math.inf # If there's no such conditional distribution in the model, return -infinity
                break

        return log_prob_sum

    def getCorpusPerplexity(self, testCorpus):
        '''
        Calculate the perplexcity of the testcorpus based on the model from the current corpus
        Input testCorpus is the list of sentences
        Output is a float number representing the perplexity of the input corpus
        '''
        import math
        log_prob_sum = 0
        word_count = 0
        for sentence in testCorpus:
            log_prob_sum += self.getSentenceLogProbability(sentence)
            word_count += len(sentence) - 1 # Don't take '<s>' as the word count

        return math.exp(-log_prob_sum / word_count)


In [17]:
if __name__=='__main__':
    sanityCheck('bigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -10.3450917073 	Your log prob.: -10.3450917073 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -9.2464794186 	Your log prob.: -9.2464794186 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'Classic', 'few', 'parts', 'of', 'the', 'game', 'allowed', 'f

In [18]:
if __name__=='__main__':
    sanityCheckFullDataset('bigram')


--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'He', 'was', '<UNK>', 'at', '<UNK>', 'College', ',', 'Hobart', ',', 'and', '<UNK>', 'in', '1932', '.', '</s>']
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'Despite', 'being', 'a', 'rare', 'Grade', '9', 'player', 'on', 'the', 'senior', 'team', ',', 'he', 'was', 'one', 'of', 'the', 'Knights', "'", 'two', 'leading', 'rushers', 'that', 'year', '.', '</s>']
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'Burke', "'s", 'total', 'was', 'a', 'school', 'record', 'for', 'the', 'Big', 'Ten', 'Conference', 'Men', "'s", 'Basketball', 'Tournament', '.', '</s>']
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'The', 'route', 'turns', 'to', 'the', 'northeast', ',', 'passing', 'near', 'the', '<UNK>', 'Leaf', 'Lakes', 'residential', 'development', ',', 'before', 'coming', 'to', 'an', 'interchange', 'with', 'US', '322', '(', 'Black', 'Horse

In [19]:
if __name__=='__main__':
    runModel('bigram')

--------- 5 sentences from your model ---------
Log Probability: -81.92438345043252 	Sentence: ['<s>', 'FRELIMO', 'and', '212', '@.@', '1', 'seconds', 'up', 'about', 'the', 'operation', 'within', 'the', 'central', '@-@', 'director', '.', '</s>']
Log Probability: -25.713749518017064 	Sentence: ['<s>', 'Between', '1940', 'while', 'strengthening', '.', '</s>']
Log Probability: -324.25078362611964 	Sentence: ['<s>', 'The', 'French', ',', 'Fuel', '@-@', 'president', 'of', 'War', 'II', 'of', 'the', 'University', 'of', 'a', '<UNK>', 'and', 'endearingly', 'flawed', ',', 'and', 'the', 'offer', 'believing', 'Robert', 'Dudley', 'seems', 'to', 'the', 'apparent', 'response', ',', 'Jan', '<UNK>', '"', 'over', '10', ',', 'and', 'comprised', '41', 'feet', '6', 'between', '1984', ',', 'is', 'hired', 'by', 'The', 'Cooper', ',', 'and', 'his', 'life', 'of', 'a', 'large', 'donations', 'were', 'also', 'found', 'to', 'usually', 'removed', 'before', 'the', 'famous', 'classical', 'themes', 'of', 'his', 'immune

### Smoothed Bigram Model

In [20]:
class SmoothedBigramModelAD(BigramModel):
    def __init__(self, trainCorpus):
        '''
        Initialize and train the model
        Input 'trainCorpus' is a list of sentence where each sentence is a list of words.
        '''
        self.corpus = trainCorpus
        self.model, self.model_count, self.D, self.S = self.train()
        self.unigram = self.get_unirgam()

    def train(self):
        '''
        Train the model with the smoothed method
        Output is a dictionary of dictionaries that represents the smoothed distribution of the model
        '''
        import math
        unigram_model = self.get_unirgam()

        model = {}
        model_count = {}
        for sentence in self.corpus:
            for i in range(len(sentence) - 1):
                word = sentence[i]
                word_next = sentence[i + 1]
                if word in model:
                    if word_next in model[word]:
                        model[word][word_next] += 1
                    else:
                        model[word][word_next] = 1
                else:
                    model[word] = {word_next : 1}

        D, S = self.calculate_DS(model)

        # Calculate the conditional porbabilities
        for word in model:
            total_next_words = sum(model[word].values())
            model_count[word] = total_next_words
            for word_next in model[word]:
                model[word][word_next] = max(model[word][word_next] - D, 0) / total_next_words + (D / total_next_words * S[word] * unigram_model[word_next])

        return model, model_count, D, S

    def calculate_DS(self, model):
        '''
        Calculate the D and S of the model
        '''
        n1 = 0
        n2 = 0
        S = {}
        for word in model:
            S[word] = len(model[word])
            for next_word in model[word]:
                if model[word][next_word] == 1:
                    n1 += 1
                elif model[word][next_word] == 2:
                    n2 += 1

        D = n1 / (n1 + 2 * n2)

        return D, S

    def get_unirgam(self):
        '''
        Get the smoothed unigram model for the corpus
        '''
        model = {}
        total_counts = 0
        for sentence in self.corpus:
            for word in sentence:
                if word != '<s>':
                    total_counts += 1
                    if word in model:
                        model[word] += 1
                    else:
                        model[word] = 1

        number_tokens = len(model.keys())

        for word in model:
            model[word] = (model[word] + 1) / (total_counts + number_tokens)

        return model

    def getSentenceLogProbability(self, sentence):
        '''
        Calculate the conditonal log probability of the sentence
        Input is the list of tokens
        Output is the float number that is the log probability of the sentence
        '''
        import math
        D = self.D
        S = self.S
        unigram = self.unigram
        log_prob_sum = 0
        for i in range(len(sentence) - 1):
            curr_word = sentence[i]
            next_word = sentence[i+1]
            total_next_words = self.model_count[curr_word]
            if curr_word in self.model and next_word in self.model[curr_word]:
                log_prob_sum += math.log(self.model[curr_word][next_word])
            else:
                log_prob_sum += math.log(D / total_next_words * S[curr_word] * unigram[next_word])


        return log_prob_sum


In [21]:
if __name__=='__main__':
    sanityCheck('smoothed-bigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -16.355820202 	Your log prob.: -16.355820202 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -76.0026113319 	Your log prob.: -76.0026113319 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -74.2346475108 	Your log prob.: -74.2346475108 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -47.2885760372 	Your log prob.: -47.2885760372 	 PASSED 	 ['<s>', 'Classic', 'few', 'pa

In [22]:
if __name__=='__main__':
    sanityCheckFullDataset('smoothed-bigram')


--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -61.3754065648 	Your log prob.: -61.3754065648 	 PASSED 	 ['<s>', 'He', 'was', '<UNK>', 'at', '<UNK>', 'College', ',', 'Hobart', ',', 'and', '<UNK>', 'in', '1932', '.', '</s>']
Correct log prob.: -141.9754903887 	Your log prob.: -141.9754903887 	 PASSED 	 ['<s>', 'Despite', 'being', 'a', 'rare', 'Grade', '9', 'player', 'on', 'the', 'senior', 'team', ',', 'he', 'was', 'one', 'of', 'the', 'Knights', "'", 'two', 'leading', 'rushers', 'that', 'year', '.', '</s>']
Correct log prob.: -107.0849366076 	Your log prob.: -107.0849366076 	 PASSED 	 ['<s>', 'Burke', "'s", 'total', 'was', 'a', 'school', 'record', 'for', 'the', 'Big', 'Ten', 'Conference', 'Men', "'s", 'Basketball', 'Tournament', '.', '</s>']
Correct log prob.: -168.4944718788 	Your log prob.: -168.4944718788 	 PASSED 	 ['<s>', 'The', 'route', 'turns', 'to', 'the', 'northeast', ',', 'passing', 'near', 'the', '<UNK>', 'Leaf', 'Lakes', 'residential', 'development', ',', 'b

In [23]:
if __name__=='__main__':
    runModel('smoothed-bigram')

--------- 5 sentences from your model ---------
Log Probability: -194.67978113056623 	Sentence: ['<s>', 'Despite', 'being', 'transferred', 'to', 'the', 'world', 'around', 'him', 'and', 'the', 'nearby', 'and', 'his', 'historical', 'periods', 'of', 'Oldham', ',', 'to', 'the', 'penis', ')', 'is', 'often', "'", '"', ',', 'one', 'of', 'the', 'MGB', 'turned', 'down', 'before', 'being', 'overly', 'dramatic', 'swing', ',', '"', 'it', 'has', 'the', '1970s', '.', '</s>']
Log Probability: -78.16105380040541 	Sentence: ['<s>', 'The', 'series', 'of', '2006', ',', 'overly', 'sentimental', 'and', 'under', '20', ',', 'they', 'supported', 'by', 'Jose', '<UNK>', '.', '</s>']
Log Probability: -190.33510793872836 	Sentence: ['<s>', 'Baby', 'Just', 'as', 'she', 'never', 'be', 'traveling', 'over', 'to', 'begin', 'until', 'her', 'children', 'as', 'a', 'sequence', 'and', ',', 'resulting', 'in', 'the', 'previous', 'location', 'of', 'World', 'War', ',', '<UNK>', 'all', 'over', 'the', 'east', 'along', 'the', 'ad